# Quantum Binary Adders
Quantum FRI, Fall 2018

In [1]:
import qiskit
import numpy as np
import math
from HW5supplemental import *

### Calculating Fourier Transforms

Let $N = 2^n$. Let $\omega$ be an $N$'th root of unity:
$$ \omega = e^{\frac{2\pi i}{N}} \hspace{1cm} \omega^N = 1$$

The following property of roots of unity is useful:
$$\sum_{x = 0}^{N-1} \omega^x = 0 \hspace{5mm}\to\hspace{5mm} \sum_{x=0}^{N-1} \omega^{xy} = 0 \text{ if } y\neq 0 \text{ mod }N, \text{ else } N $$

$$\sum_{x = 0}^{N-1} \omega^{x f(y)} |y\rangle \propto \sum_{\substack{f(y) = 0\\ \text{mod }N}}^{y \text{ where}} |y\rangle$$

This comes in handy when studying how the QFT behaves. In particular, we can use it to calculate the quantum fourier transform $U_\text{QFT}$ acts on some states. For example:
$$U_\text{QFT}|x\rangle = \frac{1}{\sqrt{N}} \sum^{2^n-1}_{y=0}  \omega^{xy} |y\rangle$$

$$U_\text{QFT} \frac{1}{\sqrt{N}} \sum_{x=0}^N \omega^{x} |x\rangle = \frac{1}{N} \sum_{y=0}^N \sum_{x=0}^N \omega^{(1+y)x} |y\rangle \propto \sum_{\substack{1+y=0\\\text{mod }N}}^{y \text{ where}} |y\rangle = |N-1\rangle $$


### 1. Quantum Fourier Transform

q is a list of quantum bits storing an integer $x \in \{0 ... (2^n-1)\}$ where $n$ = len(q) in little-endian format. Call this $n$-qubit state $|x\rangle$.


Implement $U_\text{QFT}$ on this register. See lecture 10, slide 11.

In [2]:
def qft(qc, q):
    
    n = len(q)
    
    for bit in range(n-1, -1, -1):
        qc.h(q[bit])
        for bit2 in range(bit-1, -1, -1):
            qc.cu1(2*np.pi*2**(-(bit-bit2+1)), q[bit2], q[bit])
            
    for l in range(int(np.floor(n/2))):
        qc.swap(q[l], q[n-1-l])

### 2. Inverse Quantum Fourier Transform

As can be readily computed:
$$U_\text{QFT}^\dagger|x\rangle = \frac{1}{\sqrt{N}} \sum^{N-1}_{y=0}  \omega^{-xy} |y\rangle$$

Calculate:
$$U^\dagger_\text{QFT} \frac{1}{\sqrt{N}} \sum_{x=0}^{N-1} \omega^{x} |x\rangle = ??$$

Implement qft_inv in the placeholder below.

In [3]:
def qft_inv(qc, q):
    n = len(q)
    
    for l in range(int(np.floor(n/2))):
        qc.swap(q[l], q[n-1-l])
    
    for j in range(n):
        for k in range(j):
            qc.cu1(-2*np.pi*2**(-(j-k+1)), q[k], q[j])
        qc.h(q[j])

### 3. Problem Removed

### 4. General purpose quantum adder

Implement a function that applies the unitary:

$$U_\text{add}|x\rangle_\text{input}|y\rangle_\text{output}= |x\rangle_\text{input}|y + x \text{ mod }2^m\rangle_\text{output}$$

where $x$ is an $n$-bit integer and $y$ is an $m$-bit integer. 

Use the Fourier transform trick from lecture:
$$U_\text{add} =U^\dagger_\text{qft} U_\text{phase} U_\text{qft}$$

$$\text{where }  U_\text{phase}|x\rangle|y\rangle = \omega^{xy} |x\rangle |y\rangle $$

In [4]:
def add(qc, qs_input, qs_output):
    qft(qc, qs_output)
    N = 2**(len(qs_output))
    
    for i in range(len(qs_input)):
        for j in range(len(qs_output)):
            qc.cu1(2 * math.pi * N**(-1) * 2**(i + j), qs_input[i], qs_output[j])
    
    qft_inv(qc, qs_output)

def test2(add):
    n = 3
    m = 2
    for x in range(2**n):
        for y in range(2**m):
            qc, c, qs_input, qs_output = makeIntegerCircuit(n, x, m, y)
            
            add(qc, qs_input, qs_output)
            
            
            x_out, y_out = measureIntegerCircuit(qc,c,qs_input, qs_output)
            print("U_add |"+str(x)+">|"+str(y)+"> = |"+str(x_out)+">|"+str(y_out)+">", \
                  "WRONG!" if (y_out != (x+y) % 2**m or x != x_out) else "Correct!")

test2(add)

U_add |0>|0> = |0>|0> Correct!
U_add |0>|1> = |0>|1> Correct!
U_add |0>|2> = |0>|2> Correct!
U_add |0>|3> = |0>|3> Correct!
U_add |1>|0> = |1>|1> Correct!
U_add |1>|1> = |1>|2> Correct!
U_add |1>|2> = |1>|3> Correct!
U_add |1>|3> = |1>|0> Correct!
U_add |2>|0> = |2>|2> Correct!
U_add |2>|1> = |2>|3> Correct!
U_add |2>|2> = |2>|0> Correct!
U_add |2>|3> = |2>|1> Correct!
U_add |3>|0> = |3>|3> Correct!
U_add |3>|1> = |3>|0> Correct!
U_add |3>|2> = |3>|1> Correct!
U_add |3>|3> = |3>|2> Correct!
U_add |4>|0> = |4>|0> Correct!
U_add |4>|1> = |4>|1> Correct!
U_add |4>|2> = |4>|2> Correct!
U_add |4>|3> = |4>|3> Correct!
U_add |5>|0> = |5>|1> Correct!
U_add |5>|1> = |5>|2> Correct!
U_add |5>|2> = |5>|3> Correct!
U_add |5>|3> = |5>|0> Correct!
U_add |6>|0> = |6>|2> Correct!
U_add |6>|1> = |6>|3> Correct!
U_add |6>|2> = |6>|0> Correct!
U_add |6>|3> = |6>|1> Correct!
U_add |7>|0> = |7>|3> Correct!
U_add |7>|1> = |7>|0> Correct!
U_add |7>|2> = |7>|1> Correct!
U_add |7>|3> = |7>|2> Correct!


### 5. General purpose quantum subtracter
Implement a function that applies the unitary:

$$U_\text{add}|x\rangle_\text{input}|y\rangle_\text{output}= |x\rangle_\text{input}|y - x \text{ mod }2^n\rangle_\text{output}$$


In [5]:
def subtract(qc, qs_input, qs_output):
    qft(qc, qs_output)
    
    N = 2**(len(qs_output))
    for i in range(len(qs_input)):
        for j in range(len(qs_output)):
            qc.cu1(-2 * math.pi * N**(-1) * 2**(i + j), qs_input[i], qs_output[j])
    
    qft_inv(qc, qs_output)

def test3(subtract):
    n = 3
    m = 2
    for x in range(2**n):
        for y in range(2**m):
            qc, c, qs_input, qs_output = makeIntegerCircuit(n, x, m, y)
            
            subtract(qc, qs_input, qs_output)
            
            x_out, y_out = measureIntegerCircuit(qc,c,qs_input, qs_output)
            print("U_subtract |"+str(x)+">|"+str(y)+"> = |"+str(x_out)+">|"+str(y_out)+">", \
                  "WRONG!" if (y_out != (y-x) % 2**m or x != x_out) else "Correct!")

test3(subtract)

U_subtract |0>|0> = |0>|0> Correct!
U_subtract |0>|1> = |0>|1> Correct!
U_subtract |0>|2> = |0>|2> Correct!
U_subtract |0>|3> = |0>|3> Correct!
U_subtract |1>|0> = |1>|3> Correct!
U_subtract |1>|1> = |1>|0> Correct!
U_subtract |1>|2> = |1>|1> Correct!
U_subtract |1>|3> = |1>|2> Correct!
U_subtract |2>|0> = |2>|2> Correct!
U_subtract |2>|1> = |2>|3> Correct!
U_subtract |2>|2> = |2>|0> Correct!
U_subtract |2>|3> = |2>|1> Correct!
U_subtract |3>|0> = |3>|1> Correct!
U_subtract |3>|1> = |3>|2> Correct!
U_subtract |3>|2> = |3>|3> Correct!
U_subtract |3>|3> = |3>|0> Correct!
U_subtract |4>|0> = |4>|0> Correct!
U_subtract |4>|1> = |4>|1> Correct!
U_subtract |4>|2> = |4>|2> Correct!
U_subtract |4>|3> = |4>|3> Correct!
U_subtract |5>|0> = |5>|3> Correct!
U_subtract |5>|1> = |5>|0> Correct!
U_subtract |5>|2> = |5>|1> Correct!
U_subtract |5>|3> = |5>|2> Correct!
U_subtract |6>|0> = |6>|2> Correct!
U_subtract |6>|1> = |6>|3> Correct!
U_subtract |6>|2> = |6>|0> Correct!
U_subtract |6>|3> = |6>|1> C